In [1]:
import numpy as np
import random
import math

In [2]:
def avg_var_cal(R_T, R_E, Es_dBm, Noise_Var_E, Noise_Var_B, rho_E, rho_B):
    R_S = R_T - R_E
    Es = 10**(Es_dBm/10)/1000
    sigma_E = math.sqrt(Noise_Var_E)
    sigma_B = math.sqrt(Noise_Var_B)
    avg_SNR_B = (2*Es) / (sigma_B)^2
    avg_SNR_E = (2*Es) / (sigma_E)^2
    Noise_Var_E_v = 1 - rho_E**2
    Noise_Var_B_v = 1 - rho_B**2
    return avg_SNR_B, avg_SNR_E, Noise_Var_B_v, Noise_Var_E_v

In [13]:
def TS_cha_gen(iteration, rho_B, rho_E, Noise_Var_B_v, Noise_Var_E_v, avg_SNR_B, avg_SNR_E, R_T, R_E):
    count_ConOutage_LML = []
    count_ConOutage_ZF  = []
    count_ConOutage_DF  = []

    count_SecOutage_LML = []
    count_SecOutage_ZF  = []
    count_SecOutage_DF  = []

    count_LML_LML = np.zeros(1,iteration)
    count_LML_DF = np.zeros(1,iteration)
    count_DF_LML = np.zeros(1,iteration)
    count_DF_DF = np.zeros(1,iteration)
    
    for k in range(iteration):
        ####################################
        # Time Selective Channel Generation
        ####################################
        H_B1 = np.divide(np.random.randn(2,1) + 
                np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
        H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) +
                 1j*np.random.randn(2,1)), math.sqrt(Noise_Var_B_v)), math.sqrt(2))
        H_B3 = np.c_[H_B1, H_B2]
        H_B3_JML = np.c_[H_B1, H_B1]
        H_B = ([H_B3[0,0], H_B3[1,0]],
                [np.conjugate(H_B3[1,1]), -np.conjugate(H_B3[0,1])])
        G_B = np.multiply(H_B.T, H_B)
        H_B_JML = ([H_B3_JML(0,0), H_B3_JML(1,0)],
                [np.conjugate(H_B3_JML[1,1]), -np.conjugate(H_B3_JML[0,1])])
        G_B_JML= H_B_JML.T*H_B_JML

        H_E1 = np.divide(np.random.randn(2,1) + 
                np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
        H_E2 = np.multiply(rho_E, H_E1) + np.divide(np.multiply((np.random.randn(2,1) +
                 1j*np.random.randn(2,1)), math.sqrt(Noise_Var_E_v)), math.sqrt(2))
        H_E3 = np.c_[H_E1, H_E2]
        H_E3_JML = np.c_[H_E1, H_E1]
        H_E = ([H_E3[0,0], H_E3[1,0]],
                [np.conjugate(H_E3[1,1]), -np.conjugate(H_E3[0,1])])
        G_E = np.multiply(H_E.T, H_E)
        H_E_JML = ([H_E3_JML(0,0), H_E3_JML(1,0)],
                [np.conjugate(H_E3_JML[1,1]), -np.conjugate(H_E3_JML[0,1])])
        G_E_JML= H_E_JML.T*H_E_JML

        ############################################
        # Effective rerceived SNR for each antenna
        ############################################
        # LML at Bob
        B_Eff_SNR_Ant1_LML = avg_SNR_B / ((1-rho_B**2)*avg_SNR_B + 2) * G_B[0,0]
        B_Eff_SNR_Ant2_LML = avg_SNR_B / ((1-rho_B**2)*avg_SNR_B + 2) * G_B[1,1]

        # LML at Eve
        E_Eff_SNR_Ant1_LML = avg_SNR_E / ((1-rho_E**2)*avg_SNR_E + 2) * G_E[0,0]
        E_Eff_SNR_Ant2_LML = avg_SNR_E / ((1-rho_E**2)*avg_SNR_E + 2) * G_E[1,1]

        # ZF at Bob
        B_zeta = math.sqrt(G_B[0,0]*G_B[1,1] - abs(G_B[0,1])**2);   
        B_Eff_SNR_Ant1_ZF = B_zeta**2 / (2*G_B[1,1]) * avg_SNR_B
        B_Eff_SNR_Ant2_ZF = B_zeta**2 / (2*G_B[0,0]) * avg_SNR_B
        
        # ZF at Eve
        E_zeta = math.sqrt(G_E[0,0]*G_E[1,1] - abs(G_E[0,1])**2)
        E_Eff_SNR_Ant1_ZF = E_zeta**2 / (2*G_E[1,1]) * avg_SNR_E
        E_Eff_SNR_Ant2_ZF = E_zeta**2 / (2*G_E[0,0]) * avg_SNR_E
        
        # DF at Bob
        B_zeta = math.sqrt(G_B[0,0]*G_B[1,1] - abs(G_B[0,1])**2)   
        B_Eff_SNR_Ant1_DF = B_zeta**2 / (2*G_B[1,1]) * avg_SNR_B  # ZF
        B_Eff_SNR_Ant2_DF = avg_SNR_B / 2 * G_B_JML[1,1]          # JML
        
        # DF at Eve
        E_zeta = math.sqrt(G_E[0,0]*G_E[1,1] - abs(G_E[0,1])**2)
        E_Eff_SNR_Ant1_DF = E_zeta**2 / (2*G_E[1,1]) * avg_SNR_E   # ZF
        E_Eff_SNR_Ant2_DF = avg_SNR_E / 2 * G_E_JML[1,1]           # JML

        C_B_LML = 1/2*( np.log2(1+B_Eff_SNR_Ant1_LML) + np.log2(1+B_Eff_SNR_Ant2_LML) )  # Capacity of LML at Bob
        C_B_ZF  = 1/2*( np.log2(1+B_Eff_SNR_Ant1_ZF)  + np.log2(1+B_Eff_SNR_Ant2_ZF)  )  # Capacity of ZF at Bob
        C_B_DF  = 1/2*( np.log2(1+B_Eff_SNR_Ant1_DF)  + np.log2(1+B_Eff_SNR_Ant2_DF)  )  # Capacity of DF at Bob

        C_E_LML = 1/2*( np.log2(1+E_Eff_SNR_Ant1_LML) + np.log2(1+E_Eff_SNR_Ant2_LML) )  # Capacity of LML at Eve
        C_E_ZF  = 1/2*( np.log2(1+E_Eff_SNR_Ant1_ZF)  + np.log2(1+E_Eff_SNR_Ant2_ZF)  )  # Capacity of ZF at Eve
        C_E_DF  = 1/2*( np.log2(1+E_Eff_SNR_Ant1_DF)  + np.log2(1+E_Eff_SNR_Ant2_DF)  )  # Capacity of DF at Eve

        count_ConOutage_LML[k] = (C_B_LML <= R_T)
        count_ConOutage_ZF[k]  = (C_B_ZF <= R_T)
        count_ConOutage_DF[k]  = (C_B_DF <= R_T)

        count_SecOutage_LML[k] = (C_E_LML > R_E)
        count_SecOutage_ZF[k]  = (C_E_ZF > R_E)
        count_SecOutage_DF[k]  = (C_E_DF > R_E)
    
    ConOutage_LML = np.sum(count_ConOutage_LML)/iteration
    ConOutage_ZF  = np.sum(count_ConOutage_ZF) /iteration
    ConOutage_DF  = np.sum(count_ConOutage_DF) /iteration

    SecOutage_LML  = np.sum(count_SecOutage_LML) /iteration
    SecOutage_ZF   = np.sum(count_SecOutage_ZF) /iteration
    SecOutage_DF   = np.sum(count_SecOutage_DF) /iteration

    return ConOutage_LML, ConOutage_ZF, ConOutage_DF, SecOutage_LML, SecOutage_ZF, SecOutage_DF

In [12]:
rho_B = 0.95
Noise_Var_B = 1
Noise_Var_B_v = 1 - rho_B**2

H_B1 = np.divide(np.random.randn(2,1) + np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) + 1j*np.random.randn(2,1)),
        math.sqrt(Noise_Var_B_v)), math.sqrt(2))
H_B3 = np.c_[H_B1, H_B2]
H_B3_JML = np.c_[H_B1, H_B1]
H_B = [[H_B3_JML[0,0], H_B3_JML[1,0]],
         [np.conjugate(H_B3_JML[1,1]), -np.conjugate(H_B3_JML[0,1])]]
print(np.shape(H_B))

(2, 2)
